In [1]:
import time, pyperclip
start_i = time.time()

import pandas as pd
from tqdm import tqdm
from datatools.snowflake.Querys import run_sql_query
from datatools.helpers.Utils import run_time

In [2]:
archivos = ['af','us','ac','ap','au','ah','an','am','at']

In [3]:
snow_database   = "vp_informacion"
snow_schema     = "outliers"

In [4]:
%%time

resp = dict()

for a in tqdm(archivos):
    sql = open(f"./querys/dataset/{a}.sql", mode="r", encoding="utf8").read()
    try:
        pyperclip.copy(sql)
    except:
        pass
    table = f"df_{a}"
    resp[a] = run_sql_query(sql=sql, table=table, database=snow_database, schema=snow_schema)

100%|██████████| 9/9 [15:34<00:00, 103.84s/it]

CPU times: total: 7.73 s
Wall time: 15min 34s


In [8]:
dict((k,v["status"].iloc[0]) for k,v in resp.items())

{'af': 'Table DF_AF successfully created.',
 'us': 'Table DF_US successfully created.',
 'ac': 'Table DF_AC successfully created.',
 'ap': 'Table DF_AP successfully created.',
 'au': 'Table DF_AU successfully created.',
 'ah': 'Table DF_AH successfully created.',
 'an': 'Table DF_AN successfully created.',
 'am': 'Table DF_AM successfully created.',
 'at': 'Table DF_AT successfully created.'}

In [9]:
sql = f"""SELECT 'at' as tabla, count(DISTINCT id_carga) AS cargas, count(*) AS reg, count(DISTINCT concat_ws('-',id_carga,factura)) AS facturas FROM {snow_database}.{snow_schema}.df_at
UNION ALL
SELECT 'am', count(DISTINCT id_carga) AS cargas, count(*) AS reg, count(DISTINCT concat_ws('-',id_carga,factura)) AS facturas FROM {snow_database}.{snow_schema}.df_am
UNION ALL
SELECT 'an', count(DISTINCT id_carga) AS cargas, count(*) AS reg, count(DISTINCT concat_ws('-',id_carga,factura)) AS facturas FROM {snow_database}.{snow_schema}.df_an
UNION ALL
SELECT 'au', count(DISTINCT id_carga) AS cargas, count(*) AS reg, count(DISTINCT concat_ws('-',id_carga,factura)) AS facturas FROM {snow_database}.{snow_schema}.df_au
UNION ALL
SELECT 'ap', count(DISTINCT id_carga) AS cargas, count(*) AS reg, count(DISTINCT concat_ws('-',id_carga,factura)) AS facturas FROM {snow_database}.{snow_schema}.df_ap
UNION ALL
SELECT 'ac', count(DISTINCT id_carga) AS cargas, count(*) AS reg, count(DISTINCT concat_ws('-',id_carga,factura)) AS facturas FROM {snow_database}.{snow_schema}.df_ac
UNION ALL
SELECT 'af', count(DISTINCT id_carga) AS cargas, count(*) AS reg, count(DISTINCT concat_ws('-',id_carga,factura)) AS facturas FROM {snow_database}.{snow_schema}.df_af
UNION ALL
SELECT 'us', count(DISTINCT id_carga) AS cargas, count(*) AS reg, NULL AS facturas FROM {snow_database}.{snow_schema}.df_us"""
aaa = run_sql_query(sql)
aaa

,TABLA,CARGAS,REG,FACTURAS
0,at,205685,2099569,2099569.0
1,am,224340,2480503,2480503.0
2,an,9573,33717,33717.0
3,au,76092,478996,478996.0
4,ap,277757,5647970,5647970.0
5,ac,242519,4222144,4222144.0
6,af,388438,10541692,10541692.0
7,us,388427,388427,NaN


In [11]:
run_time(start_i)

Tiempo de ejecucion total: 16.04min.
Fecha de ejecucion: 2024-12-16 09:44:01.103293


# Dataset

In [13]:
a = "dataset"

try:
    resp
except:
    resp = dict()

In [14]:
%%time
sql = open(f"./querys/dataset/{a}.sql", mode="r", encoding="utf8").read()
try:
    pyperclip.copy(sql)
except:
    pass
resp[a] = run_sql_query(sql=sql, table=a, database=snow_database, schema=snow_schema)
print(f"{snow_database}.{snow_schema}.{a}", resp[a]["status"][0])

vp_informacion.outliers.dataset Table DATASET successfully created.
CPU times: total: 797 ms
Wall time: 37.9 s


EDA dataset

In [15]:
from datatools.snowflake.Conn import load_snowflake_params
from snowflake.snowpark import Session

In [16]:
params_snow = load_snowflake_params()
session = Session.builder.configs(params_snow).create()
session

In [17]:
DF = session.table(f"{snow_database}.{snow_schema}.{a}")
DF.show(5)

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [18]:
desc = pd.DataFrame(DF.describe().collect())
desc = desc.set_index("SUMMARY").T
desc["count"] = desc["count"].astype(int)
desc

SUMMARY,min,stddev,count,max,mean
ID,00000230D6FD9CB5B56A99D25FDB7E54-1368091,None,10541692,FFFFFFC31AB6C83140B4591702468633-993797,None
FACTURA,00000230D6FD9CB5B56A99D25FDB7E54,None,10541692,FFFFFFC31AB6C83140B4591702468633,None
ID_CARGA,185690.0,395716.592811,10541692,2461450.0,919989.429907
PRESTADOR,009056BA9CC7ABB424251C0F257B3642,None,10541692,FFC81C0EC1DEFA9F959DE691C0D3DF7F,None
PERIODO_FIN_ANO,2020.0,0.755683,10541692,2022.0,2021.236405
...,...,...,...,...,...
P50_MED_NOPBS_CANTIDAD,0.0,529.294661,10541692,851400.0,1.990783
MED_NOPBS_VALOR,0.0,1312113.851649,10541692,1694309058.0,39607.356426
AVG_MED_NOPBS_VALOR,0.0,268868.498447,10541692,115284794.0,8646.003388
STD_MED_NOPBS_VALOR,0.0,42621.985801,10541692,61010544.005275,497.846088


In [19]:
desc.loc["VALOR_COPAGO":]

SUMMARY,min,stddev,count,max,mean
VALOR_COPAGO,0.0,38894.952851,10541692,24506568.0,998.867731
VALOR_COMISION,0.0,5694.528228,10541692,7685000.0,61.378287
VALOR_DESCUENTOS,0.0,57102.681549,10541692,82450120.0,314.445298
VALOR_FACTURA,1.0,15072829.078649,10541692,3615690000.0,929292.86387
PERSONAS_UNICAS,0.0,972.216437,10541692,216297.0,521.149141
...,...,...,...,...,...
P50_MED_NOPBS_CANTIDAD,0.0,529.294661,10541692,851400.0,1.990783
MED_NOPBS_VALOR,0.0,1312113.851649,10541692,1694309058.0,39607.356426
AVG_MED_NOPBS_VALOR,0.0,268868.498447,10541692,115284794.0,8646.003388
STD_MED_NOPBS_VALOR,0.0,42621.985801,10541692,61010544.005275,497.846088


In [20]:
%%time 

df = DF.to_pandas()
fName = "./dataset/dataset.parquet"
df.to_parquet(fName)
print(fName, df.shape)


./dataset/dataset.parquet (10541692, 116)
CPU times: total: 2min 43s
Wall time: 3min 38s
